In [21]:
import tensorflow as tf
import pandas as pd
tf.__version__

'2.13.0'

In [22]:
X_train = pd.read_csv("../input_data/CSVs/dota2Train.csv", header=None)
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
0,-1,223,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,152,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,131,2,2,0,0,0,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
3,1,154,2,2,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,-1,171,2,3,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# extract first column (win condition - target variable) from dataframe
y_train = X_train.pop(0)
# Replace all -1s with 0s (0 is loss, 1 is win)
y_train = y_train.replace(-1,0)
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: 0, dtype: int64

In [24]:
X_test = pd.read_csv("../input_data/CSVs/dota2Test.csv", header=None)

X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
0,-1,223,8,2,0,-1,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
1,1,227,8,2,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
2,-1,136,2,2,1,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,227,2,2,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,184,2,3,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Preprocess the target in the same way as the training dataset
y_test = X_test.pop(0)
y_test = y_test.replace(-1,0)
y_test.head()

0    0
1    1
2    0
3    1
4    1
Name: 0, dtype: int64

In [26]:
# Ensure setup matches example to verify results
tf.random.set_seed(8)
# Create a new model
model = tf.keras.Sequential()
# Fully connected layer with 512 nodes, with input matching dataset - use ReLU for activation
fc1 = tf.keras.layers.Dense(512, input_shape=(116,), activation='relu')
# Fully connected hidden ReLU layers
fc2 = tf.keras.layers.Dense(512, activation='relu')
fc3 = tf.keras.layers.Dense(128, activation='relu')
fc4 = tf.keras.layers.Dense(128, activation='relu')
# Output layer, with sigmoid activation to ensure that output is between 0 and 1 for categorisation
fc5 = tf.keras.layers.Dense(1, activation='sigmoid')

In [27]:
# Add all the layers to the model
model.add(fc1)
model.add(fc2)
model.add(fc3)
model.add(fc4)
model.add(fc5)
# Print a summary of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               59904     
                                                                 
 dense_11 (Dense)            (None, 512)               262656    
                                                                 
 dense_12 (Dense)            (None, 128)               65664     
                                                                 
 dense_13 (Dense)            (None, 128)               16512     
                                                                 
 dense_14 (Dense)            (None, 1)                 129       
                                                                 
Total params: 404865 (1.54 MB)
Trainable params: 404865 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(0.001)
# Compile the model using loss function Binary cross entropy (two categories) and Adam optimiser
model.compile(optimizer=optimizer, loss=loss)

In [29]:
# Train the model over 5 epochs
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
2896/2896 [==============================] - 20s 6ms/step - loss: 0.6995
Epoch 2/5
2896/2896 [==============================] - 18s 6ms/step - loss: 0.6709
Epoch 3/5
2896/2896 [==============================] - 19s 6ms/step - loss: 0.6670
Epoch 4/5
2896/2896 [==============================] - 19s 6ms/step - loss: 0.6660
Epoch 5/5
2896/2896 [==============================] - 19s 6ms/step - loss: 0.6652


In [30]:
# Test the model by predicting the win conditions of the testing dataset
preds = model.predict(X_test)
# print the last 5 predictions
preds[:5]

322/322 [==============================] - 1s 2ms/step


array([[0.45307103],
       [0.49221542],
       [0.6186302 ],
       [0.60425043],
       [0.35144934]], dtype=float32)